<a href="https://colab.research.google.com/github/justi-lai/Business_News_Analytics/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import requests
import json
import numpy as np
import pandas as pd
import string


In [28]:
apikey = '602a1f9dab464b98b1628cf062fb6b27'
url = 'https://newsapi.org/v2/top-headlines'

params = {
    "country": "us",
    "category": "business",
    "apiKey": apikey
}

response = requests.get(url, params)

if response.status_code == 200:
    print("Success")
else:
    print("Error")

train_data = response.json()

print(response.json()['articles'][0]['title'])

splits = {'train': 'sent_train.csv', 'validation': 'sent_valid.csv'}
df = pd.read_csv("hf://datasets/zeroshot/twitter-financial-news-sentiment/" + splits["train"])

data = df.to_numpy()

Success
Down 45% In a Week, Super Micro Computer (SMCI) Stock Turns Negative on the Year - TipRanks


In [38]:
train_data_size = 0.80
valid_data_size = 0.10
testing_size = 0.10

train_data = data[:int(train_data_size * len(data))]
valid_data = data[int(train_data_size * len(data)):int((train_data_size + valid_data_size) * len(data))]
test_data = data[int((train_data_size + valid_data_size) * len(data)):]

vocab = []
for i in train_data:
  temp = i[0].translate(str.maketrans('', '', string.punctuation))
  for j in temp.split():
    word = j.lower()
    if 'http' in word:
      continue
    if word not in vocab:
      vocab.append(word)

print(len(train_data))
print(len(valid_data))
print(len(test_data))
print(len(vocab))

7634
954
955
15702


In [39]:
print(vocab)

['bynd', 'jpmorgan', 'reels', 'in', 'expectations', 'on', 'beyond', 'meat', 'ccl', 'rcl', 'nomura', 'points', 'to', 'bookings', 'weakness', 'at', 'carnival', 'and', 'royal', 'caribbean', 'cx', 'cemex', 'cut', 'credit', 'suisse', 'jp', 'morgan', 'weak', 'building', 'outlook', 'ess', 'btig', 'research', 'cuts', 'neutral', 'fnko', 'funko', 'slides', 'after', 'piper', 'jaffray', 'pt', 'fti', 'technipfmc', 'downgraded', 'berenberg', 'but', 'called', 'top', 'pick', 'deutsche', 'bank', 'gm', 'loses', 'a', 'bull', 'hold', 'gtt', 'cowen', 'market', 'perform', 'hnhaf', 'hnhpd', 'aapl', 'trendforce', 'iphone', 'estimate', 'foxconn', 'delay', 'hog', 'moodys', 'warns', 'harleydavidson', 'hxl', 'citing', 'aero', 'ties', 'wells', 'slashes', 'hexcel', 'i', 'intelsat', 'raymond', 'james', 'krg', 'compass', 'point', 'sell', 'lk', 'muddy', 'waters', 'goes', 'short', 'luckin', 'coffee', 'mant', 'mantech', 'ahead', 'of', 'difficult', 'comps', 'mdco', 'oppenheimer', 'mplx', 'mpc', 'potential', 'dilution', '

In [31]:
class SentimentClassifier(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, output_dim, num_layers):
    super(SentimentClassifier, self).__init__()
    self.embedding = torch.Embedding(len(vocab), embedding_dim)
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers)
    self.fc = nn.Linear(hidden_dim, output_dim)

$BYND - JPMorgan reels in expectations on Beyond Meat https://t.co/bd0xbFGjkT
0
